In [ ]:
import cv2, tensorflow, os, shap
import numpy as np
from tqdm                    import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

: 

In [8]:
#讀取狗勾ㄉ照片
for i in range(1, 16, 1):
    img = cv2.imread('/Users/huangweikai/Desktop/PROJECTS/opencv_practice/archive/training_set/dogs/dog.'+str(i)+'.jpg')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [9]:
#讀取貓貓ㄉ照片
for i in range(1, 16, 1):
    img = cv2.imread('/Users/huangweikai/Desktop/PROJECTS/opencv_practice/archive/training_set/cats/cat.'+str(i)+'.jpg')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [9]:
# This part we train our model without ImageDataGenerator(不做資料擴增的意思)
class_names = ['cats','dogs']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}
nb_classes = len(class_names)
IMAGE_SIZE = (64, 64)

class_names_label

{'cats': 0, 'dogs': 1}

In [21]:
def load_data():
    datasets = ['archive/training_set', 'archive/test_set']#資料夾
    output = []
    
    # Iterate through training and test sets
    for dataset in datasets:
        
        images = []
        labels = []
        
        print("Loading {}".format(dataset))
        
        # Iterate through each folder corresponding to a category
        for folder in os.listdir(dataset):
            if folder == ".DS_Store":
                continue
            label = class_names_label[folder]
            
            # Iterate through each image in our folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                
                # Get the path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)
                if file == "desktop.ini":
                    break
                if file == ".DS_Store":
                    continue
                # Open and resize the img
                # print(img_path)
                image = cv2.imread(img_path)
                # print(image.shape)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                #cv讀照片，顏色莫認為BGR，需轉為RGB，錯誤表示黑白或已轉
                image = cv2.resize(image, IMAGE_SIZE) 
                
                # Append the image and its corresponding label to the output
                images.append(image)
                labels.append(label)
                
        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')   
        
        output.append((images, labels))

    return output

In [22]:
(train_images, train_labels), (test_images, test_labels) = load_data()

Loading archive/training_set


100%|██████████| 4001/4001 [00:10<00:00, 376.03it/s]


Loading archive/test_set


100%|██████████| 1012/1012 [00:02<00:00, 409.11it/s]


[[[[245. 215. 224.]
   [237. 202. 204.]
   [239. 198. 196.]
   ...
   [152. 118. 103.]
   [108.  82.  74.]
   [ 98.  77.  72.]]

  [[230. 195. 198.]
   [218. 173. 176.]
   [202. 153. 156.]
   ...
   [115.  86.  78.]
   [102.  83.  86.]
   [100.  85.  78.]]

  [[203. 147. 148.]
   [176. 127. 130.]
   [100.  67.  65.]
   ...
   [106.  83.  78.]
   [ 97.  81.  79.]
   [107.  85.  80.]]

  ...

  [[137.  47. 134.]
   [143.  53. 140.]
   [133.  43. 130.]
   ...
   [164. 148. 149.]
   [157. 141. 148.]
   [151. 134. 140.]]

  [[137.  47. 134.]
   [135.  46. 132.]
   [137.  47. 134.]
   ...
   [132. 116. 118.]
   [141. 125. 133.]
   [143. 126. 132.]]

  [[138.  50. 137.]
   [130.  49. 133.]
   [131.  38. 127.]
   ...
   [143. 127. 128.]
   [144. 129. 129.]
   [153. 137. 140.]]]


 [[[148. 142. 180.]
   [150. 144. 182.]
   [152. 146. 184.]
   ...
   [114.  98.  99.]
   [113.  97.  98.]
   [110.  94.  95.]]

  [[148. 142. 180.]
   [153. 147. 185.]
   [155. 149. 187.]
   ...
   [115.  99. 100.]
 

In [23]:
train_images = train_images / 255.0 
test_images = test_images / 255.0

In [33]:
model = Sequential()
model.add(Convolution2D(32 , 3, 3,  input_shape = (64, 64, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size  = (2,2)))
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))

In [34]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [1]:
# The function ImageDataGenerator augments your image by iterating through image as your CNN is getting ready to process that image
from keras.preprocessing.image import ImageDataGenerator

image_datagen = ImageDataGenerator(rescale = 1./255,
                                   horizontal_flip=True,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2, 
                                   validation_split = 0.2)

train_set = image_datagen.flow_from_directory(  'Images/all_set/',
                                                 shuffle = True,
                                                 target_size = (224, 224),
                                                 batch_size  = 16,
                                                 subset="training",
                                                 class_mode  = 'categorical'
                                                 )
test_set = image_datagen.flow_from_directory('Images/all_set/',
                                             shuffle = True,
                                            target_size = (224, 224),
                                            batch_size  = 16,
                                            subset="validation",
                                            class_mode  = 'categorical'
                                            )

2022-11-01 21:36:46.835452: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


251

In [38]:
model.fit(  train_set,
            steps_per_epoch = len(train_set),
            epochs = 1,
            validation_data = test_set,
            validation_steps = len(test_set))

251/251 [==============================] - 37s 146ms/step - loss: 0.5868 - accuracy: 0.6916 - val_loss: 0.5903 - val_accuracy: 0.7000


In [30]:
model.fit(train_images,train_labels,
            validation_data = (test_images,test_labels),
            batch_size  = 32,
            epochs = 1
            )

251/251 [==============================] - 3s 10ms/step - loss: 0.6456 - accuracy: 0.5003 - val_loss: 0.6417 - val_accuracy: 0.5002


In [11]:
model.save('cat_dog.h5')

In [16]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

In [30]:
img = image.load_img('/Users/huangweikai/Desktop/PROJECTS/opencv_practice/archive/test_set/cats/cat.4004.jpg',target_size=(64,64))
x = image.img_to_array(img)
print(x.shape)
x = np.expand_dims(x,axis = 0)

img_data = preprocess_input(x)
print(img_data.shape)
classes = model.predict(img_data)

print(classes)

(64, 64, 3)
(1, 64, 64, 3)
1/1 [==============================] - 0s 34ms/step
[[1.]]
